In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xg 

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset, drop_first=True, dtype=int)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [8]:
dependent=dataset[['charges']]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(independent, dependent, test_size=1/3, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [16]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
param_grid={'booster':['gbtree', 'dart'], 'device':['cpu','gpu']}
grid=GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree', 'dart'],
                         'device': ['cpu', 'gpu']},
             verbose=3)

In [17]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

In [18]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)

In [19]:
print("The R_score value for best parameter:{}".format(grid.best_params_),r_score)

The R_score value for best parameter:{'booster': 'gbtree', 'device': 'cpu'} 0.8238804936408997


In [20]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_device,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.103638,0.001147,0.002629,0.002354,gbtree,cpu,"{'booster': 'gbtree', 'device': 'cpu'}",0.744289,0.719668,0.829738,0.7814,0.718137,0.758646,0.042281,1
1,0.110677,0.010990,0.003422,0.004300,gbtree,gpu,"{'booster': 'gbtree', 'device': 'gpu'}",0.744289,0.719668,0.829738,0.7814,0.718137,0.758646,0.042281,1
2,1.026479,0.019333,0.010267,0.000230,dart,cpu,"{'booster': 'dart', 'device': 'cpu'}",0.744289,0.719668,0.829738,0.7814,0.718137,0.758646,0.042281,3
3,1.022505,0.028580,0.010765,0.000688,dart,gpu,"{'booster': 'dart', 'device': 'gpu'}",0.744289,0.719668,0.829738,0.7814,0.718137,0.758646,0.042281,3


In [21]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 46
BMI: 38
Children: 4
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [22]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[24623.758]
